<a href="https://colab.research.google.com/github/szkics/GerLangReconstruct/blob/main/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [26]:
!pip uninstall googletrans

Uninstalling googletrans-3.0.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/googletrans-3.0.0-py3.7.egg
Proceed (y/n)? y
  Successfully uninstalled googletrans-3.0.0


In [ ]:
!pip install googletrans==3.1.0a0

In [10]:
from googletrans import Translator
detector = Translator()
english_sentence = 'The sea is a cold place.'
german_sentence = translator.translate(english_sentence, src='en',dest='de')

print(german_sentence.text)

Das Meer ist ein kalter Ort.


In [23]:
import nltk
from nltk import tokenize
nltk.download('punkt')

list_of_english_sentences = []
file = open('/content/gdrive/MyDrive/gerlangreconstruct/english_tales.txt','r').read()
list_of_english_sentences = tokenize.sent_tokenize(file)
print(len(list_of_english_sentences))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2651


In [24]:
list_of_german_sentences = []
for english_sentence in list_of_english_sentences:
  german_sentence = translator.translate(english_sentence, src='en',dest='de')
  list_of_german_sentences.append(german_sentence.text)
print(len(list_of_german_sentences))

2651


In [39]:
list_of_english_sentences[160]

'But she never hit on the right one.'

In [40]:
list_of_german_sentences[160]#.text

'Aber sie hat nie das Richtige getroffen.'

In [ ]:
input_word_list = []
target_word_list = []
with open("/content/gdrive/MyDrive/gerlangreconstruct/german_english_dictionary.csv") as f:
    for line in f:
      splitted_line = line.split(",")
      input_word = splitted_line[0].lower()
      input_word_to_append = ''.join(e for e in input_word if e.isalnum() and not e.isdigit())
      input_word_list.append(input_word_to_append)
      target_word = splitted_line[1].lower()
      target_word_to_append = ''.join(e for e in target_word if e.isalnum() and not e.isdigit())
      target_word_list.append(target_word_to_append)
len(input_word_list)

257663

In [ ]:
len(target_word_list)
input_word_list = input_word_list[:1000]
target_word_list = target_word_list[:1000]
len(target_word_list)

1000

In [ ]:
import collections

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1759446718371640052
]


In [ ]:
german_sentences = input_word_list
english_sentences = target_word_list

In [ ]:
for sample_i in range(5):
    print('German sample {}:  {}'.format(sample_i + 1, german_sentences[sample_i]))
    print('English sample {}:  {}\n'.format(sample_i + 1, english_sentences[sample_i]))

German sample 1:  aale
English sample 1:  eels

German sample 2:  aalfell
English sample 2:  eelskin

German sample 3:  aalfischer
English sample 3:  eeler

German sample 4:  aalfänger
English sample 4:  eeler

German sample 5:  aalfänger
English sample 5:  sniggler



In [ ]:
def count_letters(input_word_list):
  input_letter_counter = {}
  alphabet = {}
  unique_index = 0
  for word in input_word_list:
    for letter in list(word):
      if letter in input_letter_counter:
        input_letter_counter[letter] = input_letter_counter[letter] + 1
      else:
        input_letter_counter[letter] = 1
      if letter not in alphabet:
        alphabet[letter] = unique_index
        unique_index = unique_index + 1
  return input_letter_counter, alphabet

input_letter_counter, input_alphabet = count_letters(german_sentences)
target_letter_counter, output_alphabet = count_letters(english_sentences)

In [ ]:
input_letter_counter

{'a': 1583,
 'b': 1305,
 'c': 244,
 'd': 376,
 'e': 1260,
 'f': 313,
 'g': 622,
 'h': 321,
 'i': 420,
 'j': 11,
 'k': 180,
 'l': 510,
 'm': 121,
 'n': 720,
 'o': 149,
 'p': 45,
 'r': 596,
 's': 354,
 't': 513,
 'u': 417,
 'v': 28,
 'w': 21,
 'x': 2,
 'y': 3,
 'z': 51,
 'ß': 10,
 'ä': 85,
 'é': 1,
 'ö': 10,
 'ü': 64}

In [ ]:
input_alphabet

{'a': 0,
 'b': 22,
 'c': 6,
 'd': 21,
 'e': 2,
 'f': 3,
 'g': 11,
 'h': 7,
 'i': 4,
 'j': 16,
 'k': 24,
 'l': 1,
 'm': 13,
 'n': 10,
 'o': 17,
 'p': 15,
 'r': 8,
 's': 5,
 't': 14,
 'u': 19,
 'v': 18,
 'w': 20,
 'x': 27,
 'y': 25,
 'z': 23,
 'ß': 28,
 'ä': 9,
 'é': 29,
 'ö': 12,
 'ü': 26}

In [ ]:
def tokenize(word, alphabet):
  vectorized = []
  for letter in list(word):
    vectorized.append(alphabet[letter])
  return vectorized

tokenized_word = tokenize("fahrrad", input_alphabet)

In [ ]:
tokenized_word

In [ ]:
def pad(x):
  y = x
  i = 0
  number_of_zeros = 42-len(x)
  while (i < number_of_zeros):
      y.append(0)
      i = i + 1
  return y
padded_word = pad(tokenized_word)

In [ ]:
padded_word

In [ ]:
def preprocess(x, y, input_alphabet, target_alphabet):

    preprocess_x = []
    preprocess_y = []

    for word in x:
      preprocess_word = tokenize(word, input_alphabet)
      preprocess_x.append(preprocess_word)

    for word in y:
      preprocess_word = tokenize(word, target_alphabet)
      preprocess_y.append(preprocess_word)

    pad_list_x = []
    pad_list_y = []

    for vectorized_word in preprocess_x:
      pad_list_x.append(pad(vectorized_word))

    for vectorized_word in preprocess_y:
      pad_list_y.append(pad(vectorized_word))

    preprocess_x = pad_list_x
    preprocess_y = pad_list_y

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    # preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y

preproc_german_sentences, preproc_english_sentences = preprocess(german_sentences, english_sentences, input_alphabet, output_alphabet)


In [ ]:
import tensorflow as tf

preproc_english_sentences = preproc_english_sentences[:400]
X_train = np.array(preproc_english_sentences)
preproc_english_sentences = preproc_english_sentences[:400]
y_train = np.array(preproc_english_sentences)
# y_train = y_train.reshape(*y_train.shape, 1)
X_dev = tf.constant([3.0, 1.0, 2.0])
y_dev = tf.constant([0.0, 2.0, 1.0])

In [ ]:
BATCH_SIZE, HIDDEN_SIZE = 32, 16
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 100

model = Sequential()
model.add(
    GRU(
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(42, 1),
        unroll=True
    )
)
model.add(Dense(42))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

for iteration in range(NUM_ITERATIONS):

    print("=" * 50)
    if (iteration % 10 == 0):
      print("Iteration #: %d" % (iteration))
    model.fit(X_train[:, :, np.newaxis], y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

In [ ]:
pred = model.predict(X_train[:, :, np.newaxis], verbose=0)[0]

In [ ]:
pred

array([1.5920916e-01, 4.6190623e-02, 6.3079035e-01, 1.0576850e-01,
       2.5791464e-02, 1.5141212e-02, 1.5139827e-02, 1.6823906e-03,
       2.8500427e-04, 1.3725962e-06, 2.8847420e-09, 9.0505614e-15,
       2.9691087e-19, 8.1305163e-26, 1.8133100e-36, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [ ]:
np.shape(X_train)

(400, 42)

In [ ]:
np.shape(X_train[:, :, np.newaxis])

(400, 42, 1)

In [ ]:
y_train

array([[ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       ...,
       [ 8, 19, 15, ...,  0,  0,  0],
       [ 8, 19, 15, ...,  0,  0,  0],
       [15,  4, 12, ...,  0,  0,  0]])

In [ ]:
word = pred * 100
word_int = word.astype(int)
arr = np.delete(word_int, np.where(word_int == 0))

In [ ]:
arr

array([15,  4, 63, 10,  2,  1,  1])

In [ ]:
word = ""
for number in arr:
  for letter, _number in input_alphabet.items():
    if _number == number:
        word = word + letter

In [ ]:
word

'pinell'

In [ ]:
word = ""
for number in arr:
  for letter, _number in output_alphabet.items():
    if _number == number:
        word = word + letter

In [ ]:
word

'difsll'

In [ ]:
arr = X_train[42]

In [ ]:
word = ""
for number in arr:
  for letter, _number in output_alphabet.items():
    if _number == number:
        word = word + letter

In [ ]:
word

'horribleeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'

In [ ]:
arr = y_train[42]
german_word = ""
for number in arr:
  for letter, _number in input_alphabet.items():
    if _number == number:
        german_word = german_word + letter

In [ ]:
german_word

'mgcciulaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [ ]:
pred = model.predict(X_train[:, :, np.newaxis], verbose=0)[42]

In [ ]:
pred

array([1.5920916e-01, 4.6190623e-02, 6.3079035e-01, 1.0576850e-01,
       2.5791464e-02, 1.5141212e-02, 1.5139827e-02, 1.6823906e-03,
       2.8500427e-04, 1.3725962e-06, 2.8847420e-09, 9.0505614e-15,
       2.9691087e-19, 8.1305163e-26, 1.8133100e-36, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [ ]:
word = pred * 100
word_int = word.astype(int)
# arr = np.delete(word_int, np.where(word_int == 0))

In [ ]:
predicted_word = ""
for number in arr:
  for letter, _number in input_alphabet.items():
    if _number == number:
        predicted_word = predicted_word + letter

In [ ]:
predicted_word

'mgcciulaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'